In [3]:
# SERVICE_ACCOUNT_PATH = "creds/google__sa.json"
# TASKS_SHEET_ID = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
# TASKS_SHEET_NAME = "Conversations_Batch_3"
# REVIEWS_SHEET_NAME = "Reviews"

In [4]:

# import pandas as pd

# from src.sheets_utils import download_sheet_as_df

# df_completed = download_sheet_as_df(SERVICE_ACCOUNT_PATH, TASKS_SHEET_ID, "Conversations_Batch_3")
# df_reviews = download_sheet_as_df(SERVICE_ACCOUNT_PATH, TASKS_SHEET_ID, "Reviews")

In [9]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd

def list_colab_notebooks(service_account_path, folder_id):
    # Authenticate with the service account
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = service_account.Credentials.from_service_account_file(service_account_path, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    notebooks = []
    page_token = None

    try:
        # Query to search for Google Colab notebooks with the '.ipynb' extension
        query = f"'{folder_id}' in parents and name contains '.ipynb'"

        while True:
            response = service.files().list(
                q=query,
                spaces='drive',
                fields='nextPageToken, files(id, name, webViewLink)',
                pageToken=page_token).execute()

            for item in response.get('files', []):
                colab_link = f"https://colab.research.google.com/drive/{item['id']}"
                notebooks.append({
                    'name': item['name'],
                    'id': item['id'],
                    'webViewLink': item['webViewLink'],
                    'colabLink': colab_link
                })

            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break

    except Exception as e:
        print("An error occurred:", e)

    # Convert the list of notebooks to a DataFrame
    df = pd.DataFrame(notebooks, columns=['name', 'id', 'webViewLink', 'colabLink'])
    return df

service_account_path = './creds/google__sa.json'
folder_id = '1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'

df_colab = list_colab_notebooks(service_account_path, folder_id)
print(df_colab)

                                   name                                 id  \
0          trees__modification__2.ipynb  1zxiQD6kBrgojEK8mNW5panISHbyIuCN5   
1          trees__modification__1.ipynb  1AfLZOOr4XIxpmzhGISzzXoATjE8cLeUb   
2          trees__modification__0.ipynb  1KWsFiH4pmfZOjzUcklFEdL-oywzWIHVm   
3                stacks__query__4.ipynb  14ySY4SUSPjy91ifJUgjrP6UyYY3j1bJK   
4                stacks__query__3.ipynb  1HoQ-0gPWj9iQyxvs1GGLrfkX4zs6PKm2   
5                stacks__query__2.ipynb  1L1FAd7ETmwcN6Oq2f3Uwl2eiFU7F5UDm   
6                stacks__query__1.ipynb  1-lWTHifHX75-W-GBFiYbFHsHZfNTnM0P   
7                stacks__query__0.ipynb  1b07DKfbsFzT_HUWLYYkjIfoUjUfAHqOl   
8         stacks__modification__2.ipynb  1iD2M6WWnOBytYIajcR4Ct9dJVxzGCRnl   
9         stacks__modification__1.ipynb  1GkkglGVB3CbPBL9LJ14xAp5GesBDvjKe   
10        stacks__modification__0.ipynb  1fsUYt6bf_Rc0gFcAf0wJu9d9m4IUqW4P   
11               queues__query__4.ipynb  1xXGyiBHOyzklvwfw_Zjj-y

In [19]:
df_colab['colabLink']

0     https://colab.research.google.com/drive/1zxiQD...
1     https://colab.research.google.com/drive/1AfLZO...
2     https://colab.research.google.com/drive/1KWsFi...
3     https://colab.research.google.com/drive/14ySY4...
4     https://colab.research.google.com/drive/1HoQ-0...
5     https://colab.research.google.com/drive/1L1FAd...
6     https://colab.research.google.com/drive/1-lWTH...
7     https://colab.research.google.com/drive/1b07DK...
8     https://colab.research.google.com/drive/1iD2M6...
9     https://colab.research.google.com/drive/1Gkkgl...
10    https://colab.research.google.com/drive/1fsUYt...
11    https://colab.research.google.com/drive/1xXGyi...
12    https://colab.research.google.com/drive/1jQ9Gy...
13    https://colab.research.google.com/drive/140lJ2...
14    https://colab.research.google.com/drive/1Yhgjv...
15    https://colab.research.google.com/drive/1WAZug...
16    https://colab.research.google.com/drive/1lhcX8...
17    https://colab.research.google.com/drive/1G

In [18]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import json

def list_json_files(service_account_path, folder_id, file_name='Problems.json'):
    # Authenticate with the service account
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = service_account.Credentials.from_service_account_file(service_account_path, scopes=SCOPES)
    service = build('drive', 'v3', credentials=creds)

    # Query to search for JSON files in the specified folder
    query = f"'{folder_id}' in parents and name='{file_name}'"

    # Execute the query and get the list of files
    files = []
    page_token = None
    while True:
        response = service.files().list(
            q=query, 
            spaces='drive', 
            fields='nextPageToken, files(id, name)',
            pageToken=page_token
        ).execute()
        
        files.extend(response.get('files', []))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    
    return files

def download_file_contents(service, file_id):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
    fh.seek(0)
    
    return json.load(fh)

def read_problems_json(service_account_path, folder_id):
    # Authenticate and build the Google Drive service
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = service_account.Credentials.from_service_account_file(service_account_path, scopes=SCOPES)
    drive_service = build('drive', 'v3', credentials=creds)

    # Get list of 'Problems.json' files
    json_files = list_json_files(service_account_path, folder_id)

    # Merge all 'Problems.json' files into a single list
    all_problems = []
    for json_file in json_files:
        file_contents = download_file_contents(drive_service, json_file['id'])
        all_problems.extend(file_contents)
        
    return all_problems

service_account_path = './creds/google__sa.json'
folder_id = '1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'

problems = read_problems_json(service_account_path, folder_id)
len(problems)

40

# Google Sheets API Code

This Jupyter notebook code snippet is used to automate interactions with Google Sheets using the Sheets API. The code performs the following tasks:

1. **Setting Up Authentication**: It imports the required service account credentials and initializes the required API scope for accessing the Google Sheets API.

2. **Defining Spreadsheet ID**: The ID of the Google Spreadsheet we want to interact with is specified as a string.

3. **Building the Service**: With the provided credentials and the 'sheets' API service name and version, it constructs a Google API service object.

4. **Retrieving Sheet Metadata**: The code fetches metadata about the spreadsheets present in the document corresponding to `SPREADSHEET_ID`.

5. **Finding the Highest Sheet Number**: It goes through all the sheet titles retrieved earlier and uses a regular expression to find sheets named following the pattern `conversation_{number}` to determine the highest number used.

6. **Creating a New Sheet**: Once the highest sheet number is found, it is incremented by 1 to create a unique sheet name, then a new sheet is added to the spreadsheet with this new name.

7. **Writing Data**: The code prepares a set of values extracted from a variable called `problems`. It then defines a range within the newly created sheet and writes these values into that range.

8. **Executing the Update**: The prepared data is sent in an update request to the API, populating the new sheet with the values.

9. **Printing Success Message**: Finally, the code prints a message confirming the successful creation and population of the new sheet.

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import re

SERVICE_ACCOUNT_FILE = './creds/google__sa.json'

# The scopes required for the Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID of your spreadsheet
SPREADSHEET_ID = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Retrieve all sheets metadata
sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheets = sheet_metadata.get('sheets', '')

# Find the highest number in existing 'conversation' sheet names
highest_num = 0
for sheet in sheets:
    title = sheet['properties']['title']
    match = re.match(r'conversation_(\d+)', title)
    if match:
        num = int(match.group(1))
        highest_num = max(highest_num, num)

# Increment the highest number by 1 to create new sheet name
new_sheet_name = f"conversation_{highest_num + 1}"

# Create the new sheet with the new name
requests = [{
    'addSheet': {
        'properties': {
            'title': new_sheet_name
        }
    }
}]
body = {'requests': requests}
response = service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=body).execute()

# Extract the new sheet ID from response
new_sheet_id = response['replies'][0]['addSheet']['properties']['sheetId']

values = []
for problem in problems:
    values.append([
        problem["metadata"]["colab_url"],
        problem["metadata"]["topic"],
        problem["metadata"]["target_length"],
        problem["metadata"]["type"],
        problem["metadata"]["batch_idx"],
    ])

# The range where to write the data: starting from cell A2 in the new sheet
range_ = f"'{new_sheet_name}'!A2:E610"

value_body = {
    'values': values
}

# Update the sheet with the values
update_request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=range_,
    valueInputOption='RAW',
    body=value_body
)
update_response = update_request.execute()

print(f"Created and populated new sheet: {new_sheet_name}")